# Darik's School to Code Good
## Best Practices for Writing Clean, Maintainable, Bug-Free Code

Goal: make your code easier to **read, debug, reuse, and review**.

We’ll cover:

1. Variable naming
2. Proper documentation
3. Modularity & single responsibility
4. Cyclomatic complexity
5. Principle of least surprise
6. Loose coupling
7. Test-driven development (TDD)

Throughout, think about your own analysis scripts and how you’d refactor them.


## 0. Imports and helper for this notebook

We'll use a couple of helpers for examples and tests.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Callable

import math
import statistics as stats


## 1. Variable naming

> *“Code is read much more often than it is written.”*

**Bad patterns**
- Single-letter or cryptic names (`a`, `x1`, `tmp2`)
- Names that lie or are too generic (`data`, `results`, `val`)
- Inconsistent conventions (`nTrials` vs `num_trials`)

**Good patterns**
- Names show *role* and *units* where relevant
- Consistent style: `snake_case` for variables and functions in Python
- Booleans that read as a statement: `is_valid`, `has_spikes`


In [ ]:
# ❌ Hard to read and understand
def f(x, y):
    # x is a list of times
    # y is a list of vals
    s = 0
    n = 0
    for i in range(len(x)):
        if y[i] > 0:
            s += y[i]
            n += 1
    return s / n


In [ ]:
# ✅ Clearer naming and intent
def mean_positive(values: list[float]) -> float:
    """Compute the mean of strictly positive values in a list."""
    positive_values = [v for v in values if v > 0]
    return stats.fmean(positive_values)


### Your turn

Refactor this function with better variable names:

```python
def g(a, b):
    r = []
    for i in range(len(a)):
        if a[i] > b:
            r.append(a[i])
    return r
```

- Rename the function to describe *what* it does.
- Rename `a`, `b`, `r` to something meaningful.


## 2. Proper documentation

We’ll focus on **docstrings**:

- **What** the function does (one short sentence)
- **Arguments** (types, meaning, units if relevant)
- **Returns** (what and in what format)
- Edge cases, assumptions, side effects

Docstrings are for *external* communication.
Inline comments are for *local* clarification of non-obvious steps.


In [ ]:
def normalize_trace(trace, m=None, s=None):
    if m is None:
        m = sum(trace) / len(trace)
    if s is None:
        s = (sum((x - m)**2 for x in trace) / (len(trace) - 1))**0.5
    return [(x - m)/s for x in trace]


In [ ]:
def zscore_trace(
    trace: list[float],
    mean: float | None = None,
    std: float | None = None,
) -> list[float]:
    """
    Z-score a 1D signal.

    Parameters
    ----------
    trace : list of float
        Signal values (e.g., fluorescence over time).
    mean : float, optional
        Pre-computed mean. If None, computed from `trace`.
    std : float, optional
        Pre-computed standard deviation. If None, computed from `trace`.

    Returns
    -------
    list of float
        Z-scored signal with zero mean and unit variance.
    """
    if mean is None:
        mean = stats.fmean(trace)
    if std is None:
        # Sample standard deviation
        std = stats.stdev(trace)
    return [(x - mean) / std for x in trace]


### Your turn

- Add a docstring to `mean_positive` from above.
- Include: what it does, expected input, what happens if there are no positives.


## 3. Modularity & Single Responsibility

A function should do **one thing** and do it well.

Benefits:
- Easier to test
- Easier to reuse in a different context
- Easier to change internal implementation without breaking callers


In [ ]:
# ❌ Everything in one blob
def analyze_reaction_times(file_path: str, threshold: float = 1.0) -> float:
    """Return fraction of fast trials from a CSV file of reaction times."""
    # load CSV
    with open(file_path) as f:
        lines = f.readlines()
    rts = []
    for line in lines[1:]:  # skip header
        rt = float(line.strip().split(",")[1])
        rts.append(rt)
    # clean
    rts = [rt for rt in rts if rt > 0]
    # compute metric
    fast = [rt for rt in rts if rt < threshold]
    return len(fast) / len(rts)


In [ ]:
# ✅ Decomposed into focused pieces
def load_reaction_times(file_path: str) -> list[float]:
    """Load reaction times from a CSV with header, reaction time in column 2."""
    rts: list[float] = []
    with open(file_path) as f:
        next(f)  # skip header
        for line in f:
            _, rt_str = line.strip().split(",")
            rts.append(float(rt_str))
    return rts


def filter_valid_reaction_times(reaction_times: list[float]) -> list[float]:
    """Filter out invalid reaction times (<= 0)."""
    return [rt for rt in reaction_times if rt > 0]


def fraction_fast_trials(
    reaction_times: list[float],
    threshold: float = 1.0,
) -> float:
    """Compute fraction of trials with reaction time below threshold (s)."""
    valid_rts = filter_valid_reaction_times(reaction_times)
    fast_trials = [rt for rt in valid_rts if rt < threshold]
    return len(fast_trials) / len(valid_rts)


### Your turn

- Imagine you have a function that:
  1. Loads imaging data
  2. Motion-corrects it
  3. Extracts ΔF/F
  4. Plots the result

Sketch how you would split that into 3–5 smaller functions with single responsibilities.
(No need to implement fully here; just function names + docstrings.)


## 4. Cyclomatic complexity

Cyclomatic complexity ~ **number of independent paths** through your code.

High complexity:
- Many `if`/`elif` branches
- Deeply nested loops / conditionals
- Harder to test and reason about

Rule of thumb: keep functions simple enough that you can hold the logic in your head.


In [ ]:
# ❌ Dense branching; hard to test exhaustively
def classify_trial(rt: float, accuracy: bool, difficulty: str) -> str:
    """
    Classify a single trial as 'good', 'ok', or 'bad'.
    """
    if difficulty == "easy":
        if accuracy and rt < 0.7:
            return "good"
        elif accuracy and rt < 1.0:
            return "ok"
        else:
            return "bad"
    elif difficulty == "hard":
        if accuracy and rt < 1.2:
            return "good"
        elif accuracy and rt < 1.6:
            return "ok"
        else:
            return "bad"
    else:
        if accuracy and rt < 1.0:
            return "good"
        elif accuracy and rt < 1.4:
            return "ok"
        else:
            return "bad"


In [ ]:
def difficulty_thresholds(difficulty: str) -> tuple[float, float]:
    """
    Return (good_threshold, ok_threshold) in seconds for each difficulty.
    """
    if difficulty == "easy":
        return 0.7, 1.0
    if difficulty == "hard":
        return 1.2, 1.6
    # default
    return 1.0, 1.4


def classify_trial(rt: float, accuracy: bool, difficulty: str) -> str:
    """
    Classify a trial as 'good', 'ok', or 'bad'.
    """
    good_threshold, ok_threshold = difficulty_thresholds(difficulty)

    if not accuracy:
        return "bad"
    if rt < good_threshold:
        return "good"
    if rt < ok_threshold:
        return "ok"
    return "bad"


### Your turn

- Where did we reduce cyclomatic complexity in `classify_trial`?
- How would you further simplify if the classification rules changed often?


## 5. Principle of Least Surprise

Code should behave in a way that is **unsurprising** to someone who knows the language and your domain.

Surprising behavior:
- Functions that mutate arguments when callers expect a copy
- Functions with hidden global state / side effects
- Functions that return different *types* depending on inputs


In [ ]:
# ❌ Surprising: mutates default list; default argument is shared between calls
def collect_values(x, values=[]):
    values.append(x)
    return values

print(collect_values(1))
print(collect_values(2))  # surprise: previous call's data is still there


In [ ]:
# ✅ No shared mutable default
def collect_values(x, values=None):
    """
    Append `x` to an existing list or create a new one.
    """
    if values is None:
        values = []
    values.append(x)
    return values

print(collect_values(1))
print(collect_values(2))


### Your turn

- Think of a function in your own code that might surprise a new lab member.
  - Does it mutate inputs?
  - Does it read from / write to global variables?
- Write down one change you could make to reduce surprise.


## 6. Loose coupling

**Coupling** = how strongly different parts of your system depend on each other.

We want:
- Modules that know as little as possible about each other's internals
- Clear *interfaces* (function signatures, return types) instead of hidden shared state
- Easier replacement / reuse (e.g., swapping a file loader for a database loader)


In [ ]:
# ❌ Analysis function knows too much about how data is stored on disk
def compute_mean_fluorescence(mouse_id: str) -> float:
    file_path = f"./data/{mouse_id}_day1_fov1.csv"
    with open(file_path) as f:
        next(f)
        values = [float(line.strip().split(",")[0]) for line in f]
    return stats.fmean(values)


In [ ]:
# ✅ Analysis depends on an interface, not on disk layout
def compute_mean_signal(
    loader: Callable[[str], list[float]],
    session_id: str,
) -> float:
    """
    Compute mean signal using a generic loader.

    The loader is any function that takes a session_id and returns a list of floats.
    """
    signal = loader(session_id)
    return stats.fmean(signal)


def csv_signal_loader(session_id: str) -> list[float]:
    file_path = f"./data/{session_id}.csv"
    with open(file_path) as f:
        next(f)
        return [float(line.strip().split(",")[0]) for line in f]


# Later you can swap to another loader without changing compute_mean_signal:
# def hdf5_signal_loader(session_id: str) -> list[float]:
#     ...


### Your turn

- Identify a function in your work that:
  - Both **loads** data and **analyzes** it.
- Write a new function signature that would separate loading from analysis.

Example pattern:

```python
def analyze_trials(load_trials: Callable[[str], list[Trial]], experiment_id: str) -> AnalysisResult:
    ...
```


## 7. Test-Driven Development (TDD)

TDD cycle:

1. **Write a test** that describes desired behavior (it should fail).
2. **Write the minimal code** to make the test pass.
3. **Refactor** while keeping tests green.

For teaching, we’ll use the built-in `assert` statements, but in practice you’d use `pytest`.


### Step 1: Write tests first

We want a function `percent_change(old, new)` that:

- Returns percent change as a float (e.g., from 10 to 15 → 50.0)
- Raises `ValueError` if `old == 0` to avoid division by zero


In [ ]:
def test_percent_change_basic():
    assert percent_change(10, 15) == 50.0
    assert percent_change(20, 10) == -50.0
    assert percent_change(5, 5) == 0.0


def test_percent_change_zero_old():
    try:
        percent_change(0, 10)
    except ValueError:
        pass
    else:
        raise AssertionError("Expected ValueError when old == 0")

# Run tests (they will fail because percent_change is not defined yet)
try:
    test_percent_change_basic()
    test_percent_change_zero_old()
    print("All tests passed (unexpected)")
except NameError as e:
    print("As expected, tests fail before implementation:", e)


In [ ]:
def percent_change(old: float, new: float) -> float:
    """
    Compute percent change from `old` to `new`.

    Example: old=10, new=15 -> 50.0
    """
    if old == 0:
        raise ValueError("old must be non-zero to compute percent change")
    return (new - old) / old * 100.0


In [ ]:
test_percent_change_basic()
test_percent_change_zero_old()
print("All tests passed ✅")


### Your turn

- Pick a small function from earlier in this notebook (e.g., `mean_positive`).
- Write 2–3 tests for it:
  - A normal case
  - An edge case
  - A failure case (if appropriate)
- Only then modify the function to satisfy those tests.


## 8. Summary checklist

When you write or review code, ask:

1. **Names**
   - Are function and variable names descriptive?
   - Do booleans read like true/false statements?

2. **Documentation**
   - Does each function have a clear one-line summary?
   - Are arguments and return values documented (including units)?

3. **Modularity**
   - Does each function do one job?
   - Could you test it in isolation?

4. **Cyclomatic complexity**
   - Are there many deeply nested `if`/`for` blocks?
   - Can you extract helpers to simplify?

5. **Principle of least surprise**
   - Would behavior surprise a new lab member?
   - Any hidden side effects or type changes?

6. **Loose coupling**
   - Does analysis code depend directly on file paths / globals?
   - Can you introduce clear interfaces (functions, classes) instead?

7. **Tests / TDD**
   - Is there at least one test for non-trivial functions?
   - Could you write the next function by starting from tests?

---

### Optional extension

In a real project, you would also add:

- **Linting** (e.g., `ruff`, `flake8`) and **formatting** (`black`)
- **Type checking** with `mypy` or `pyright`
- A `tests/` directory with `pytest` tests
- A `README` describing how to run the code

Use this notebook as a starting point and adapt the examples to your own lab codebase.
